# In questo notebook calcolo il numero di tweet totali e medi pubblicati da ciascun utente in un anno

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import csv
import time

In [3]:
spark = SparkSession.builder.appName("frequency analysis by month 1").config("spark.executor.memory", "500m").getOrCreate()

In [4]:
df = spark.read.options(sep=",", header=True, quote="\"", escape="\"", multiline=True).csv('DatiDaAnalizzare_pt4.csv')

In [5]:
tuples_RDD = df.rdd

In [6]:
user_month_to_one_RDD = tuples_RDD.map(f=lambda e: ((e[3], (time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(e[0],'%a %b %d %H:%M:%S +0000 %Y')))[5:7], (time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(e[0],'%a %b %d %H:%M:%S +0000 %Y')))[0:4]), 1))

In [7]:
user_month_to_sum_RDD = user_month_to_one_RDD.reduceByKey(func=lambda a,b: a+b)

In [8]:
user_year_to_month_sum_RDD = user_month_to_sum_RDD.map(f=lambda e: ((e[0][0], e[0][2]), (e[1], 1)))

In [9]:
user_year_to_sum_RDD = user_year_to_month_sum_RDD.reduceByKey(func=lambda a,b: (a[0]+b[0], a[1]+b[1]))

In [10]:
user_year_to_avg_RDD = user_year_to_sum_RDD.map(f=lambda e: (e[0][0], e[0][1], e[1][0], e[1][1], e[1][0]//e[1][1]))

In [11]:
sorted_RDD = user_year_to_avg_RDD.sortBy(keyfunc=lambda x: -x[2])

In [12]:
result_RDD = sorted_RDD.map(f=lambda x: x[0] + "," + str(x[1]) + "," + str(x[2]) + "," + str(x[3]) + "," + str(x[4]))

In [13]:
result_RDD.saveAsTextFile('Pubblicazioni_annuali_pt4')